## Read files and Data summary

In [1]:
import pandas as pd
import numpy as np
from __future__ import division
import re as re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar

#### Read files and have an overview of the dataset

In [2]:
emails = pd.read_csv('emails.tsv', header=None, delimiter = '\t')
emails.columns = ['email_id', 'sent_time','email_type','email_variant','member_id']
print emails.shape
print emails.dtypes

(9607809, 5)
email_id          int64
sent_time        object
email_type       object
email_variant    object
member_id         int64
dtype: object


In [3]:
email_responses = pd.read_csv('email_responses.tsv', delimiter = '\t')

In [4]:
email_responses.shape

(1558609, 3)

In [6]:
# number of emails 
a = email_responses['email_id'].unique()
len(a)

902915

In [7]:
# initial stats of open/click/unsub
action_dist = email_responses['action'].value_counts().to_dict()
action_dist

{'click': 436201, 'open': 1099755, 'unsub': 22653}

## Data cleaning

● drop duplicates

● merge emails.tsv and email_responses.tsv

● basic summary of open/click/unsub rate

● map features 

● data transformation

● use re to extract potential info from columns

● sent_time transformation

#### drop duplicates and merge emails and emails_responses

In [8]:
open_email = email_responses.loc[email_responses['action']=='open']
open_email.columns = ['email_id','open_time','open']
open_email = open_email.drop_duplicates(subset=['email_id'],keep='last')
temp0 = emails.merge(open_email,how='left',left_on='email_id',right_on='email_id')

In [9]:
unique_open_email0 = sum(open_email.email_id.isin(emails.email_id))
unique_open_email1 = len(open_email)
print 'The number of emails that are in subset(open) of email_response.tsv but out of emails.tsv:' + ' '+ str(unique_open_email1-unique_open_email0)

The number of emails that are in subset(open) of email_response.tsv but out of emails.tsv: 36107


In [10]:
click_email = email_responses.loc[email_responses['action']=='click']
click_email.columns = ['email_id','click_time','click']
click_email = click_email.drop_duplicates(subset=['email_id'],keep='last')
temp1 = temp0.merge(click_email,how='left',on='email_id')

In [11]:
unique_click_email0 = sum(click_email.email_id.isin(emails.email_id))
unique_click_email1 = len(click_email)
print 'The number of emails that are in subset(click) of email_response.tsv but out of emails.tsv:' + ' '+ str(unique_click_email1-unique_click_email0)

The number of emails that are in subset(click) of email_response.tsv but out of emails.tsv: 5346


In [12]:
unsub_email = email_responses.loc[email_responses['action']=='unsub']
unsub_email.columns = ['email_id','unsub_time','unsub']
unsub_email = unsub_email.drop_duplicates(subset=['email_id'],keep='last')
email_sent_response = temp1.merge(unsub_email,how='left',on='email_id')

In [13]:
unique_unsub_email0 = sum(unsub_email.email_id.isin(emails.email_id))
unique_unsub_email1 = len(unsub_email)
print 'The number of emails that are in subset(unsub) of email_response.tsv but out of emails.tsv:' + ' '+ str(unique_unsub_email1-unique_unsub_email0)

The number of emails that are in subset(unsub) of email_response.tsv but out of emails.tsv: 1481


In [14]:
email_sent_response

,email_id,sent_time,email_type,email_variant,member_id,open_time,open,click_time,click,unsub_time,unsub
0,205569994,2012-09-01 00:00:07,Transactional JR Welcome Email,account_login_info_s2_v1,14802252,NaN,NaN,NaN,NaN,NaN,NaN
1,205569995,2012-09-01 00:00:07,Transactional JR Welcome Email,account_login_info_s2_v1,14802253,NaN,NaN,NaN,NaN,NaN,NaN
2,205569997,2012-09-01 00:00:07,Transactional JR Welcome Email,account_login_info_s2_v1,14802255,NaN,NaN,NaN,NaN,NaN,NaN
3,205569998,2012-09-01 00:00:07,Transactional JR Welcome Email,account_login_info_s2_v1,14802256,NaN,NaN,NaN,NaN,NaN,NaN
4,205570000,2012-09-01 00:00:07,Transactional JR Welcome Email,account_login_info_s2_v1,14802257,NaN,NaN,NaN,NaN,NaN,NaN
5,205570071,2012-09-01 00:01:07,Transactional Forgot Password Email,NaN,14613748,NaN,NaN,NaN,NaN,NaN,NaN
6,205570072,2012-09-01 00:03:06,Transactional Forgot Password Email,NaN,14802258,NaN,NaN,NaN,NaN,NaN,NaN
7,205570073,2012-09-01 00:04:07,Transactional Forgot Password Email,NaN,14802258,NaN,NaN,NaN,NaN,NaN,NaN
8,205570074,2012-09-01 00:06:06,Transactional Forgot Password Email,NaN,14802258,NaN,NaN,NaN,NaN,NaN,NaN
9,205570075,2012-09-01 00:07:07,Transactional Forgot Password Email,NaN,14802258,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# number of unique members
a = email_sent_response['member_id'].unique()
len(a)

1886779

In [ ]:
# number of unique unsubscirbes
b = email_sent_response.loc[email_sent_response['unsub'] == 'unsub']
c = b['member_id'].unique()
len(c)

#### First view of open/click/unsub rate

In [16]:
open_count = email_sent_response['open'].value_counts()
click_count = email_sent_response['click'].value_counts()
unsub_count = email_sent_response['unsub'].value_counts()
email_count = len(email_sent_response)
a = (click_count)/(email_count)
print click_count
print open_count
print unsub_count
print 'Open Rate is' +' '+ "{:.2%}".format(open_count['open']/email_count)
print 'Click Rate is' +' '+ "{:.2%}".format(click_count['click']/email_count)
print 'Unsubscribe Rate is' +' '+ "{:.2%}".format(unsub_count['unsub']/email_count)
print 'Click-Open Rate is' +' '+ "{:.2%}".format(click_count['click']/open_count['open'])

click    331118
Name: click, dtype: int64
open    765554
Name: open, dtype: int64
unsub    19698
Name: unsub, dtype: int64
Open Rate is 7.97%
Click Rate is 3.45%
Unsubscribe Rate is 0.21%
Click-Open Rate is 43.25%


In [17]:
email_sent_response['email_type'].value_counts().to_dict()

{'T plus 1': 279398,
 'T plus N': 8708618,
 'Transactional Forgot Password Email': 121880,
 'Transactional JR Welcome Email': 497913}

In [18]:
email_sent_response['open'] = email_sent_response['open'].replace('open',1)
email_sent_response['open'].fillna(0,inplace=True)
email_sent_response['open'] = email_sent_response['open'].astype(int)

email_sent_response['click'] = email_sent_response['click'].replace('click',1)
email_sent_response['click'].fillna(0,inplace=True)
email_sent_response['click'] = email_sent_response['click'].astype(int)

email_sent_response['unsub'] = email_sent_response['unsub'].replace('unsub',1)
email_sent_response['unsub'].fillna(0,inplace=True)
email_sent_response['unsub'] = email_sent_response['unsub'].astype(int)


#### Mapping email_type:

● Transactional Forgot Password Email: 0

● Transactional JR Welcome Email: 1

● T plus 1: 2

● T plus N: 3

In [19]:
mapping = {'Transactional Forgot Password Email': 0, 'Transactional JR Welcome Email': 1, 'T plus 1': 2, 'T plus N': 3}
email_sent_response['email_type'] = email_sent_response['email_type'].map(mapping).astype(int)

#### Using regular expression to separate features from email_variant

In [20]:
def get_days_joined(campaign):
    days_search = re.search('(tplus+)([+-]?\d+(?:\.\d+)?)',campaign)
    if days_search:
        return days_search.group(2)
    return 0
email_sent_response['email_variant'] = email_sent_response['email_variant'].astype('str') 
email_sent_response['days_joined'] = email_sent_response['email_variant'].apply(lambda x: get_days_joined(x))

In [21]:
def get_campaign(campaign):
    cloud_email = re.search('fixed_keyword_cloud_s1_v1',campaign)
    job_alert = re.search('job_alert_s1_v1',campaign)
    login = re.search('account_login_info_s2_v1',campaign)
    birthday = re.search('birthday_s1_v1_f2',campaign)
    birthday_job = re.search('birthday_joblist1_s1_v1_f2',campaign)
    if cloud_email:
        return 'cloud email'
    if job_alert:
        return 'job_listing'
    if login:
        return 'log_in'
    if birthday:
        return 'birthday'
    if birthday_job:
        return 'birthday_job'
    return 'password'
email_sent_response['email_kind'] = email_sent_response['email_variant'].apply(lambda x: get_campaign(x))

In [22]:
def get_opened(campaign):
    opened_before = re.search('1opened',campaign)
    if opened_before:
        return 1
    return 0
email_sent_response['opened_before'] = email_sent_response['email_variant'].apply(lambda x: get_opened(x))

In [23]:
def get_age22(campaign):
    age22 = re.search('age22+',campaign)
    if age22:
        return 1
    return 0
email_sent_response['age22'] = email_sent_response['email_variant'].apply(lambda x: get_age22(x))

In [24]:
def get_age35(campaign):
    age35 = re.search('age35+',campaign)
    if age35:
        return 1
    return 0
email_sent_response['age35'] = email_sent_response['email_variant'].apply(lambda x: get_age35(x))
cols = ['opened_before','age22','age35']
email_sent_response[cols] = email_sent_response[cols].astype(int)

#### Convert to NaN

In [25]:
cols_to_nan = ['opened_before','age22','age35']
email_sent_response.loc[email_sent_response['email_type']== 0, cols_to_nan] = np.nan
email_sent_response.loc[email_sent_response['email_type']== 1, cols_to_nan] = np.nan
email_sent_response.loc[email_sent_response['email_type']== 2, cols_to_nan] = np.nan


In [26]:
email_sent_response['days_joined'] = email_sent_response['days_joined'].astype(int)
email_sent_response['category_days_joined'] = pd.cut(email_sent_response['days_joined'], 15)

email_sent_response.loc[email_sent_response['email_type']== 0,'category_days_joined'] = np.nan
email_sent_response.loc[email_sent_response['email_type']== 1,'category_days_joined'] = np.nan
email_sent_response.loc[email_sent_response['email_type']== 2,'category_days_joined'] = np.nan

#### Extract potential info from sent_time

● create date,hour,weekday columns

● transform continuous variable to categorical variable 

In [27]:
email_sent_response["date"] = email_sent_response.sent_time.apply(lambda x : x.split()[0])
email_sent_response["hour"] = email_sent_response.sent_time.apply(lambda x : x.split()[1].split(":")[0])
email_sent_response["weekday"] = email_sent_response.date.apply(lambda dateString : calendar.day_name[datetime.strptime(dateString,"%Y-%m-%d").weekday()])

In [28]:
email_sent_response['hour'] = email_sent_response['hour'].astype(int)
email_sent_response['category_hour'] = pd.cut(email_sent_response['hour'], 4)
email_sent_response['category_hour'].value_counts()

(-0.023, 5.75]    5255033
(5.75, 11.5]      3875697
(11.5, 17.25]      311497
(17.25, 23.0]      165582
Name: category_hour, dtype: int64

#### Coercing to category type

In [29]:
categoryVarList = ['email_type','email_kind','opened_before','age35','age22']
for var in categoryVarList:
    email_sent_response[var] = email_sent_response[var].astype('category')

#### Drop unnecessary columns

In [30]:
email_sent_response = email_sent_response.drop(['email_variant'],axis=1)
email_sent_response = email_sent_response.drop(['days_joined'],axis=1)
email_sent_response = email_sent_response.drop(['hour'],axis=1)
email_sent_response = email_sent_response.drop(['sent_time'],axis=1)

In [31]:
email_sent_response.dtypes

email_id                   int64
email_type              category
member_id                  int64
open_time                 object
open                       int64
click_time                object
click                      int64
unsub_time                object
unsub                      int64
email_kind              category
opened_before           category
age22                   category
age35                   category
category_days_joined    category
date                      object
weekday                   object
category_hour           category
dtype: object

In [32]:
email_sent_response

,email_id,email_type,member_id,open_time,open,click_time,click,unsub_time,unsub,email_kind,opened_before,age22,age35,category_days_joined,date,weekday,category_hour
0,205569994,1,14802252,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
1,205569995,1,14802253,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
2,205569997,1,14802255,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
3,205569998,1,14802256,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
4,205570000,1,14802257,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
5,205570071,0,14613748,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
6,205570072,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
7,205570073,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
8,205570074,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
9,205570075,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"


In [33]:
email_sent_response.loc[(email_sent_response['unsub']==1) & (email_sent_response['click']==1)]

,email_id,email_type,member_id,open_time,open,click_time,click,unsub_time,unsub,email_kind,opened_before,age22,age35,category_days_joined,date,weekday,category_hour
65130,205641149,3,14040931,2012-09-08 05:11:55,1,2012-09-08 05:12:35,1,2012-09-08 05:15:06,1,job_listing,1.0,0.0,0.0,"(24.133, 48.267]",2012-09-01,Saturday,"(-0.023, 5.75]"
66076,205642095,3,14489345,2012-09-02 07:56:48,1,2012-09-02 07:57:32,1,2012-09-02 07:57:51,1,job_listing,0.0,1.0,0.0,"(-0.362, 24.133]",2012-09-01,Saturday,"(-0.023, 5.75]"
66116,205642135,3,14488698,NaN,0,2012-09-01 16:04:30,1,2012-09-01 16:10:15,1,cloud email,0.0,0.0,0.0,"(-0.362, 24.133]",2012-09-01,Saturday,"(-0.023, 5.75]"
66641,205642660,3,14043601,NaN,0,2012-09-06 15:43:47,1,2012-09-06 15:44:03,1,job_listing,1.0,0.0,0.0,"(24.133, 48.267]",2012-09-01,Saturday,"(-0.023, 5.75]"
70894,205646913,3,14056052,2012-09-08 12:58:53,1,2012-09-01 12:45:45,1,2012-09-08 12:59:00,1,cloud email,1.0,0.0,0.0,"(24.133, 48.267]",2012-09-01,Saturday,"(-0.023, 5.75]"
71303,205647322,3,14062912,2012-09-08 07:44:30,1,2012-09-08 07:44:53,1,2012-09-08 07:46:36,1,job_listing,1.0,0.0,0.0,"(24.133, 48.267]",2012-09-01,Saturday,"(-0.023, 5.75]"
75806,205651825,3,14382048,NaN,0,2012-09-01 22:27:02,1,2012-09-01 22:30:40,1,cloud email,0.0,1.0,0.0,"(-0.362, 24.133]",2012-09-01,Saturday,"(-0.023, 5.75]"
76699,205652718,3,14351829,2012-09-01 15:40:31,1,2012-09-01 15:40:36,1,2012-09-01 15:41:04,1,job_listing,0.0,1.0,0.0,"(-0.362, 24.133]",2012-09-01,Saturday,"(-0.023, 5.75]"
82579,205658598,3,14382429,NaN,0,2012-09-01 14:11:25,1,2012-09-01 14:13:14,1,job_listing,0.0,1.0,0.0,"(-0.362, 24.133]",2012-09-01,Saturday,"(-0.023, 5.75]"
86519,205662538,3,14355656,2012-09-01 19:04:26,1,2012-09-01 19:04:35,1,2012-09-01 19:05:20,1,cloud email,0.0,0.0,0.0,"(-0.362, 24.133]",2012-09-01,Saturday,"(-0.023, 5.75]"


In [34]:
email_kind = email_sent_response.groupby(['email_kind','age22','age35','opened_before']).size()
email_kind

email_kind   age22  age35  opened_before
cloud email  0.0    0.0    0.0              1519907
                           1.0               780283
                    1.0    0.0                60537
             1.0    0.0    0.0               374204
job_listing  0.0    0.0    1.0              2608118
                    1.0    0.0               232627
             1.0    0.0    0.0              3132906
password     0.0    0.0    0.0                   36
dtype: int64

In [ ]:
# email_sent_response.to_csv('email_sent_response.tsv', sep='\t')

## Statistics

#### Overview of email_type on open/click/unsub rate

In [35]:
# Overview of email_type on open/click/unsub rate
r_emailvar = email_sent_response[['email_type','click','open']].groupby(['email_type'], as_index=False).mean()
r_emailvar['click_open'] = r_emailvar['click']/r_emailvar['open']
a = pd.DataFrame(email_sent_response.groupby('email_type').member_id.nunique())
a = a.reset_index()
b = pd.DataFrame(email_sent_response.loc[email_sent_response['unsub']==1].groupby('email_type').member_id.nunique())
b = b.reset_index()
b['unsub'] = b['member_id']/a['member_id'] 
r_emailvar['unsub'] = b['unsub']
r_emailvar
# email_type = 1: unsub rate is high: no additional value/single CTA
# email_type = 2: unsub rate is above avg: activation/email format

,email_type,click,open,click_open,unsub
0,0,0.236520,0.285141,0.829482,0.000092
1,1,0.043236,0.124741,0.346611,0.010837
2,2,0.002276,0.108336,0.021012,0.002634
3,3,0.032167,0.073309,0.438780,0.009350


#### email_kind - open/click/unsub rate

In [36]:
# email_kind - open/click/unsub rate
r_email_kind = email_sent_response[['email_kind','click','open']].groupby(['email_kind'], as_index=False).mean()
r_email_kind['click_open'] = r_email_kind['click']/r_email_kind['open']
a = pd.DataFrame(email_sent_response.groupby('email_kind').member_id.nunique())
a = a.reset_index()
b = pd.DataFrame(email_sent_response.loc[email_sent_response['unsub']==1].groupby('email_kind').member_id.nunique())
b = b.reset_index()
b['unsub'] = b['member_id']/a['member_id'] 
r_email_kind['unsub'] = b['unsub']
# r_email_kind.to_csv('email_kind_rate.tsv', sep='\t')
r_email_kind

,email_kind,click,open,click_open,unsub
0,birthday,0.001839,0.107346,0.017128,0.002666
1,birthday_job,0.021023,0.150769,0.139438,0.001255
2,cloud email,0.037201,0.069764,0.533237,0.006347
3,job_listing,0.029862,0.074933,0.398520,0.008180
4,log_in,0.043236,0.124741,0.346611,0.010837
5,password,0.236450,0.285073,0.829435,0.000092


#### time_to_send impacts on open/click/unsub rate

In [37]:
# time_to_send impacts on open/click/unsub rate
r_hour = email_sent_response[['email_kind','email_type','category_hour','click','open']].groupby(['email_kind','category_hour'], as_index=False).mean()
r_hour['click_open'] = r_hour['click']/r_hour['open']
email_sent_response[['email_kind','email_type','category_hour','click','open']].groupby(['email_kind','category_hour'], as_index=False).size()
# r_hour.to_csv('email_sent_hour_rate.tsv', sep='\t')
# email_kind = cloud email: click rate is high between 6 to 12
# email_kind = job listing: click rate is low between 6 to 12

email_kind    category_hour 
birthday      (5.75, 11.5]       273024
birthday_job  (5.75, 11.5]         6374
cloud email   (-0.023, 5.75]    1373856
              (5.75, 11.5]      1361075
job_listing   (-0.023, 5.75]    3875179
              (5.75, 11.5]      2098472
log_in        (-0.023, 5.75]       5349
              (5.75, 11.5]        95519
              (11.5, 17.25]      253355
              (17.25, 23.0]      143690
password      (-0.023, 5.75]        649
              (5.75, 11.5]        41233
              (11.5, 17.25]       58142
              (17.25, 23.0]       21892
dtype: int64

#### weekday impacts on open/click/unsub rate

In [38]:
r_weekday = email_sent_response[['email_kind','weekday','click','open','unsub']].groupby(['weekday'], as_index=False).mean()
r_weekday['click_open'] = r_weekday['click']/r_weekday['open']
# r_weekday.loc[(r_weekday['email_kind']=='cloud email')|(r_weekday['email_kind']=='job_listing')]
a = pd.DataFrame(email_sent_response.groupby('weekday').member_id.nunique())
a = a.reset_index()
b = pd.DataFrame(email_sent_response.loc[email_sent_response['unsub']==1].groupby('weekday').member_id.nunique())
b = b.reset_index()
b['unsub'] = b['member_id']/a['member_id'] 
r_weekday['unsub'] = b['unsub']
r_weekday

,weekday,click,open,unsub,click_open
0,Friday,0.039107,0.084069,0.005184,0.465174
1,Monday,0.042066,0.094557,0.005553,0.444873
2,Saturday,0.023894,0.062214,0.004134,0.384061
3,Sunday,0.030459,0.076783,0.004930,0.396696
4,Thursday,0.041860,0.087088,0.005507,0.480660
5,Tuesday,0.039264,0.089322,0.005396,0.439580
6,Wednesday,0.029651,0.070433,0.004432,0.420984


#### days_joined impacts on open/click/unsub rate

In [39]:
# days_joined impacts on open/click/unsub rate
r_joined_days = email_sent_response[['email_kind','email_type','category_days_joined','click','open','unsub']].groupby(['email_type','email_kind','category_days_joined'], as_index=False).mean()
r_joined_days['click_open'] = r_joined_days['click']/r_joined_days['open']
r_joined_days.loc[(r_joined_days['email_type'] == 3)&(r_joined_days['email_kind'] == 'cloud email')]
r_joined_days.loc[(r_joined_days['email_type'] == 3)&(r_joined_days['email_kind'] == 'job_listing')]

,email_type,email_kind,category_days_joined,click,open,unsub,click_open
315,3,job_listing,"(-0.362, 24.133]",0.041767,0.076867,0.001898,0.543371
316,3,job_listing,"(24.133, 48.267]",0.026178,0.067421,0.001336,0.388280
317,3,job_listing,"(48.267, 72.4]",0.027246,0.085684,0.001491,0.317980
318,3,job_listing,"(72.4, 96.533]",0.021860,0.089642,0.001307,0.243861
319,3,job_listing,"(96.533, 120.667]",0.019738,0.083087,0.001053,0.237554
320,3,job_listing,"(120.667, 144.8]",0.019919,0.082104,0.001194,0.242611
321,3,job_listing,"(144.8, 168.933]",0.018820,0.076546,0.000905,0.245869
322,3,job_listing,"(168.933, 193.067]",0.017340,0.071085,0.000885,0.243931
323,3,job_listing,"(193.067, 217.2]",0.016904,0.069812,0.000794,0.242138
324,3,job_listing,"(217.2, 241.333]",0.016866,0.069651,0.000774,0.242143


In [40]:
email_sent_response

,email_id,email_type,member_id,open_time,open,click_time,click,unsub_time,unsub,email_kind,opened_before,age22,age35,category_days_joined,date,weekday,category_hour
0,205569994,1,14802252,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
1,205569995,1,14802253,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
2,205569997,1,14802255,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
3,205569998,1,14802256,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
4,205570000,1,14802257,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
5,205570071,0,14613748,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
6,205570072,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
7,205570073,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
8,205570074,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
9,205570075,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"


In [41]:
email_sent_response.groupby('category_days_joined')['member_id'].nunique()

category_days_joined
(-0.362, 24.133]      637537
(24.133, 48.267]      387076
(48.267, 72.4]        189705
(72.4, 96.533]        146008
(96.533, 120.667]     138265
(120.667, 144.8]      126049
(144.8, 168.933]      132609
(168.933, 193.067]     98630
(193.067, 217.2]      117746
(217.2, 241.333]      116555
(241.333, 265.467]     89245
(265.467, 289.6]       44194
(289.6, 313.733]       49366
(313.733, 337.867]     36475
(337.867, 362.0]       30178
Name: member_id, dtype: int64

#### campaign age22+ - open/click/unsub rate

In [42]:
# campaign age22+ - open/click/unsub rate
r_ekind_ctype_age22 = email_sent_response[['email_kind','email_type','age22','click','open','unsub']].groupby(['email_kind','age22'], as_index=False).mean()
r_ekind_ctype_age22['click_open'] = r_ekind_ctype_age22['click']/r_ekind_ctype_age22['open']

r_ekind_ctype_age22.loc[(r_ekind_ctype_age22['email_kind']=='cloud email')|(r_ekind_ctype_age22['email_kind']=='job_listing')]


,email_kind,age22,click,open,unsub,click_open
4,cloud email,0.0,0.035938,0.069688,0.001757,0.515701
5,cloud email,1.0,0.045165,0.070242,0.002146,0.642990
6,job_listing,0.0,0.021397,0.077956,0.001109,0.274478
7,job_listing,1.0,0.037538,0.072191,0.001742,0.519976


In [43]:
email_sent_response.groupby(['age22']).size()

age22
0.0    5201508
1.0    3507110
dtype: int64

In [44]:
email_sent_response.loc[email_sent_response['age22']==1,'click'].mean()

0.038351520197541566

#### campaign age35+ - open/click/unsub rate

In [45]:
r_ekind_ctype_age35 = email_sent_response[['email_kind','email_type','age35','click','open','unsub']].groupby(['email_kind','age35'], as_index=False).mean()
r_ekind_ctype_age35['click_open'] = r_ekind_ctype_age35['click']/r_ekind_ctype_age35['open']
r_ekind_ctype_age35.loc[(r_ekind_ctype_age35['email_kind']=='cloud email')|(r_ekind_ctype_age35['email_kind']=='job_listing')]

,email_kind,age35,click,open,unsub,click_open
4,cloud email,0.0,0.037392,0.070067,0.001820,0.533655
5,cloud email,1.0,0.028759,0.056362,0.001355,0.510258
6,job_listing,0.0,0.029928,0.075176,0.001438,0.398107
7,job_listing,1.0,0.028238,0.068934,0.001517,0.409641


In [46]:
email_sent_response.groupby(['age35']).size()

age35
0.0    8415454
1.0     293164
dtype: int64

In [47]:
email_sent_response.loc[email_sent_response['age35']==1,'click'].mean()

0.0283459087746108

#### campaign opened_before - open/click/unsub rate

In [48]:
r_ekind_ctype_opened = email_sent_response[['email_kind','email_type','opened_before','click','open','unsub']].groupby(['email_kind','opened_before'], as_index=False).mean()
r_ekind_ctype_opened['click_open'] = r_ekind_ctype_opened['click']/r_ekind_ctype_opened['open']
r_ekind_ctype_opened.loc[(r_ekind_ctype_opened['email_kind']=='cloud email')|(r_ekind_ctype_opened['email_kind']=='job_listing')]


,email_kind,opened_before,click,open,unsub,click_open
4,cloud email,0.0,0.043997,0.069517,0.002059,0.632887
5,cloud email,1.0,0.020176,0.070381,0.001187,0.286669
6,job_listing,0.0,0.036895,0.071966,0.001726,0.512671
7,job_listing,1.0,0.020787,0.078761,0.001072,0.263927


In [49]:
email_sent_response.groupby(['opened_before']).size()

opened_before
0.0    5320217
1.0    3388401
dtype: int64

In [50]:
email_sent_response.loc[email_sent_response['opened_before']==1,'click'].mean()

0.020646316654964983

## Net Profit

#### Drop unsub and password clicks

In [51]:
email_sent_response = email_sent_response[~((email_sent_response['unsub'] == 1) & (email_sent_response['click'] == 1))]
email_sent_response = email_sent_response[~((email_sent_response['email_type'] == 0) & (email_sent_response['click'] == 1))]
email_sent_response

,email_id,email_type,member_id,open_time,open,click_time,click,unsub_time,unsub,email_kind,opened_before,age22,age35,category_days_joined,date,weekday,category_hour
0,205569994,1,14802252,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
1,205569995,1,14802253,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
2,205569997,1,14802255,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
3,205569998,1,14802256,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
4,205570000,1,14802257,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
5,205570071,0,14613748,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
6,205570072,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
7,205570073,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
8,205570074,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
9,205570075,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"


#### Revenue/Cost/Profit

In [52]:
len(email_sent_response.loc[email_sent_response['click']==1])

298235

In [53]:
total_revenue = 0.12 * len(email_sent_response.loc[email_sent_response['click']==1])
print total_revenue
total_cost = 0.4 * 9574926/1000
print total_cost
gross_margin = (total_revenue - total_cost)
print gross_margin

35788.2
3829.9704
31958.2296


#### Revenue/Cost/Profit by email_kind

In [54]:
dict_of_email = {k: v for k, v in email_sent_response.groupby('email_kind')}

In [55]:
total_gross_margin = 0
for k in dict_of_email:        
    revenue = 0.12 * len(dict_of_email[k].loc[dict_of_email[k]['click']==1])
    cost = 0.4 * len(dict_of_email[k]['click'])/1000
    gm = revenue - cost
    ROI = gm/cost
    gm_unit = (revenue - cost)/len(dict_of_email[k]['click'])
    print 'The number of',k,'is:',len(dict_of_email[k]['click'])
    print 'The gm_unit of',k,'is:',gm_unit
    print ROI
    print 'The Total GM of', k , 'is:',gm
    total_gross_margin += gm
total_gross_margin

The number of cloud email is: 2733740
The gm_unit of cloud email is: 0.00401373356647
10.0343339162
The Total GM of cloud email is: 10972.504
The number of log_in is: 497127
The gm_unit of log_in is: 0.0046068493564
11.517123391
The Total GM of log_in is: 2290.1892
The number of birthday_job is: 6374
The gm_unit of birthday_job is: 0.00212274866646
5.30687166614
The Total GM of birthday_job is: 13.5304
The number of birthday is: 273012
The gm_unit of birthday is: -0.000184624851655
-0.461562129137
The Total GM of birthday is: -50.4048
The number of password is: 93089
The gm_unit of password is: -0.0004
-1.0
The Total GM of password is: -37.2356
The number of job_listing is: 5971584
The gm_unit of job_listing is: 0.00314316040769
7.85790101923
The Total GM of job_listing is: 18769.6464


31958.2296

#### Revenue by Opened_before campaign

In [56]:
opened_campaign = email_sent_response.loc[email_sent_response['opened_before']==1]
revenue = 0.12 * len(opened_campaign.loc[opened_campaign['click']==1])
cost = 0.4 * len(opened_campaign['click'])/1000
gm = revenue - cost
ROI = gm/cost
opened_gm_unit = (revenue - cost)/len(opened_campaign['click'])
print 'The ROI of campaign_opened is:',ROI
print 'The number of Opened_before emails is:',len(opened_campaign['click']) 
print 'The Total GM of campaign_opened is:',gm

The ROI of campaign_opened is: 5.1429010397
The number of Opened_before emails is: 3387813
The Total GM of campaign_opened is: 6969.2748


#### Revenue by age22+ campaign

In [57]:
age22_campaign = email_sent_response.loc[email_sent_response['age22']==1]
revenue = 0.12 * len(age22_campaign.loc[age22_campaign['click']==1])
cost = 0.4 * len(age22_campaign['click'])/1000
gm = revenue - cost
ROI = gm/cost
age22_gm_unit = (revenue - cost)/len(age22_campaign['click'])
print 'The ROI of campaign_age22 is:',ROI
print 'The number of age22 emails is:',len(age22_campaign['click']) 
print 'The Total GM of campaign_age22 is:',gm

The ROI of campaign_age22 is: 10.3601942177
The number of age22 emails is: 3505345
The Total GM of campaign_age22 is: 14526.422


#### Revenue by age35+ campaign

In [58]:
age35_campaign = email_sent_response.loc[email_sent_response['age35']==1]
revenue = 0.12 * len(age35_campaign.loc[age35_campaign['click']==1])
cost = 0.4 * len(age35_campaign['click'])/1000
gm = revenue - cost
ROI = gm/cost
age35_gm_unit = (revenue - cost)/len(age35_campaign['click'])
print 'The ROI of campaign_age35 is:',ROI
print 'The number of age35 emails is:',len(age35_campaign['click'])
print 'The Total GM of campaign_age35 is:',gm

The ROI of campaign_age35 is: 7.41226729268
The number of age35 emails is: 293072
The Total GM of campaign_age35 is: 868.9312


#### Revenue by weekday

In [59]:
dict_of_email_weekday = {k: v for k, v in email_sent_response.groupby('weekday')}

In [60]:
total_gm_weekday = 0
for k in dict_of_email_weekday:        
    revenue = 0.12 * len(dict_of_email_weekday[k].loc[dict_of_email_weekday[k]['click']==1])
    cost = 0.4 * len(dict_of_email_weekday[k]['click'])/1000
    gm = revenue - cost
    ROI = gm/cost
    weekday_gm_unit = (revenue - cost)/len(dict_of_email_weekday[k]['click'])
    
    print 'The ROI of',k,'is:',ROI
    print gm
    total_gm_weekday += gm
total_gm_weekday

The ROI of Monday is: 10.5377994582
5509.4524
The ROI of Tuesday is: 9.44686836672
4826.1292
The ROI of Friday is: 9.67406026122
4816.4476
The ROI of Wednesday is: 6.69208485605
3603.768
The ROI of Thursday is: 10.2288210471
4630.7264
The ROI of Sunday is: 7.51530504732
4921.638
The ROI of Saturday is: 5.59575772813
3650.068


31958.2296

In [61]:
email_sent_response.groupby('weekday').size()

weekday
Friday       1244681
Monday       1307069
Saturday     1630730
Sunday       1637205
Thursday     1131784
Tuesday      1277177
Wednesday    1346280
dtype: int64

In [62]:
email_sent_response

,email_id,email_type,member_id,open_time,open,click_time,click,unsub_time,unsub,email_kind,opened_before,age22,age35,category_days_joined,date,weekday,category_hour
0,205569994,1,14802252,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
1,205569995,1,14802253,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
2,205569997,1,14802255,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
3,205569998,1,14802256,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
4,205570000,1,14802257,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
5,205570071,0,14613748,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
6,205570072,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
7,205570073,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
8,205570074,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"
9,205570075,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]"


#### Profit by state

In [63]:
members_state = pd.read_csv('members_state.tsv', delimiter = '\t')
members_state = members_state.drop(members_state.columns[0], axis=1)
members_state.loc[members_state['member_id']==14802270]

,member_id,state
1101021,14802270,TX


In [64]:
df = email_sent_response.merge(members_state,how='left',on='member_id')
df

,email_id,email_type,member_id,open_time,open,click_time,click,unsub_time,unsub,email_kind,opened_before,age22,age35,category_days_joined,date,weekday,category_hour,state
0,205569994,1,14802252,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]",AZ
1,205569995,1,14802253,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]",VA
2,205569997,1,14802255,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]",IN
3,205569998,1,14802256,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]",CO
4,205570000,1,14802257,NaN,0,NaN,0,NaN,0,log_in,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]",CA
5,205570071,0,14613748,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]",WA
6,205570072,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]",CA
7,205570073,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]",CA
8,205570074,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]",CA
9,205570075,0,14802258,NaN,0,NaN,0,NaN,0,password,NaN,NaN,NaN,NaN,2012-09-01,Saturday,"(-0.023, 5.75]",CA


In [65]:
r_state = df[['email_kind','state','click','open']].groupby(['state'], as_index=False).mean()
r_state['click_open'] = r_state['click']/r_state['open']
r_state

,state,click,open,click_open
0,AA,0.000000,0.000000,NaN
1,AE,0.071429,0.170068,0.420000
2,AK,0.025573,0.072318,0.353617
3,AL,0.028293,0.060332,0.468955
4,AP,0.081967,0.180328,0.454545
5,AR,0.026597,0.064859,0.410071
6,AS,0.100629,0.062893,1.600000
7,AZ,0.035617,0.086239,0.413007
8,CA,0.032913,0.088586,0.371536
9,CO,0.034190,0.089561,0.381750


In [66]:
email_state = df['state'].value_counts()
email_state = pd.DataFrame(email_state)
email_state.columns = ['number of emails']
email_state['state'] = email_state.index
email_state = email_state.reset_index()
email_state = email_state.drop(email_state.columns[0], axis=1)
email_state

,number of emails,state
0,880168,CA
1,801302,TX
2,771417,FL
3,472081,GA
4,397332,IL
5,396304,NY
6,377396,OH
7,377198,NC
8,348458,PA
9,279739,MI


In [67]:
r_state = df[['email_kind','state','click','open']].groupby(['state'], as_index=False).sum()
r_state = r_state.merge(email_state,how='left',on='state')
r_state['total_gross_margin'] = (0.12*r_state['click'])-(0.0004*r_state['number of emails'])
r_state['gm_per_email'] = r_state['total_gross_margin']/r_state['number of emails']
# r_state.to_csv('profit_state.tsv', sep='\t')